# Memory Subsystem Comparison

Comprehensive comparison of memory management components:
- **mimalloc_glue** - Memory allocation performance
- **smr_ibr** - Interval-Based Reclamation
- **smr_debra** - DEBRA+ with neutralization

This notebook helps you:
- Compare IBR vs DEBRA+ under various conditions
- Understand memory bound differences (O(T²R) vs O(TR))
- Choose the right SMR scheme for your workload
- Analyze end-to-end memory subsystem performance

## 1. Setup

In [ ]:
import sys
import threading
import time
import random
from dataclasses import dataclass
from datetime import datetime
from typing import Optional, Dict, Any

try:
    from concurrent_collections import (
        MemoryProfiler, SMRProfiler, DEBRAProfiler,
        SkipListMap, config
    )
    from concurrent_collections.profiling import compare_smr_schemes
    LIBRARY_AVAILABLE = True
    print(f"concurrent_collections loaded")
    print(f"Platform: {sys.platform}")
    print(f"Python: {sys.version.split()[0]}")
except ImportError:
    LIBRARY_AVAILABLE = False
    print("Running in simulation mode")

## 2. Simulation Data

In [ ]:
if not LIBRARY_AVAILABLE:
    @dataclass
    class ComparisonResult:
        scheme: str
        duration_seconds: float
        operations: int
        ops_per_sec: float
        
        # Memory metrics
        peak_allocated_bytes: int
        peak_pending_count: int
        peak_pending_bytes: int
        
        # Reclamation metrics
        total_retired: int
        total_reclaimed: int
        reclaim_latency_p50_ns: float
        reclaim_latency_p99_ns: float
        
        # Epoch metrics
        epoch_advances: int
        safe_epoch_lag_max: int
        
        # Stall/neutralization metrics
        stall_count: int
        stall_time_ns: int
        neutralization_count: int  # DEBRA+ only
        restart_count: int  # DEBRA+ only
        
        # Theoretical bounds
        theoretical_bound: int
        bound_utilization: float

    def generate_simulated_comparison():
        """Generate simulated comparison data for IBR vs DEBRA+."""
        T, R = 8, 64  # threads, retire threshold
        
        ibr = ComparisonResult(
            scheme='ibr',
            duration_seconds=30.0,
            operations=2_400_000,
            ops_per_sec=80_000,
            peak_allocated_bytes=45_000_000,
            peak_pending_count=1_200,
            peak_pending_bytes=76_800,
            total_retired=720_000,
            total_reclaimed=719_500,
            reclaim_latency_p50_ns=8_000,
            reclaim_latency_p99_ns=250_000,
            epoch_advances=45_000,
            safe_epoch_lag_max=35,
            stall_count=12,
            stall_time_ns=800_000_000,
            neutralization_count=0,
            restart_count=0,
            theoretical_bound=3 * T * T * R,  # O(T²R)
            bound_utilization=1_200 / (3 * T * T * R),
        )
        
        debra = ComparisonResult(
            scheme='debra',
            duration_seconds=30.0,
            operations=2_350_000,
            ops_per_sec=78_333,
            peak_allocated_bytes=42_000_000,
            peak_pending_count=480,
            peak_pending_bytes=30_720,
            total_retired=705_000,
            total_reclaimed=704_800,
            reclaim_latency_p50_ns=6_000,
            reclaim_latency_p99_ns=150_000,
            epoch_advances=52_000,
            safe_epoch_lag_max=8,
            stall_count=0,
            stall_time_ns=0,
            neutralization_count=45,
            restart_count=180,
            theoretical_bound=3 * T * R,  # O(TR)
            bound_utilization=480 / (3 * T * R),
        )
        
        return {'ibr': ibr, 'debra': debra}
    
    print("Simulation classes created")

## 3. Test Configuration

In [ ]:
TEST_CONFIG = {
    'num_threads': 8,
    'duration_seconds': 30,
    'operations_per_thread': 100_000,
    'key_range': 50_000,
    'delete_ratio': 0.3,
    'stall_probability': 0.002,  # 0.2% - enough to stress SMR
    'stall_duration_ms': 10,
}

print("Test Configuration:")
for key, value in TEST_CONFIG.items():
    print(f"  {key}: {value}")

T = TEST_CONFIG['num_threads']
R = 64  # Default retire threshold
print(f"\nTheoretical Memory Bounds:")
print(f"  IBR: O(T²R) = O({T}² × {R}) = {3 * T * T * R:,} nodes")
print(f"  DEBRA+: O(TR) = O({T} × {R}) = {3 * T * R:,} nodes")
print(f"  Ratio: IBR/DEBRA+ = {T}x worse in worst case")

## 4. Run Comparison

In [ ]:
if LIBRARY_AVAILABLE:
    def workload_fn():
        """Standard workload for comparison."""
        m = SkipListMap()
        
        def worker():
            for i in range(TEST_CONFIG['operations_per_thread']):
                key = f"key_{random.randint(0, TEST_CONFIG['key_range'])}"
                
                if random.random() < TEST_CONFIG['stall_probability']:
                    time.sleep(TEST_CONFIG['stall_duration_ms'] / 1000)
                
                if random.random() < TEST_CONFIG['delete_ratio']:
                    m.pop(key, None)
                else:
                    m[key] = f"value_{i}"
        
        threads = [
            threading.Thread(target=worker)
            for _ in range(TEST_CONFIG['num_threads'])
        ]
        for t in threads:
            t.start()
        for t in threads:
            t.join()
    
    print("Running comparison (this may take a minute)...")
    print("\n--- Testing IBR ---")
    config.smr_scheme = 'ibr'
    # ... run IBR test ...
    
    print("\n--- Testing DEBRA+ ---")
    config.smr_scheme = 'debra'
    # ... run DEBRA+ test ...
    
    # In actual implementation, use compare_smr_schemes()
    results = compare_smr_schemes(
        workload=workload_fn,
        schemes=['ibr', 'debra'],
        duration=TEST_CONFIG['duration_seconds'],
    )
    print("\nComparison complete!")
else:
    print("Using simulated comparison data...")
    results = generate_simulated_comparison()

## 5. Side-by-Side Comparison

In [ ]:
ibr = results['ibr']
debra = results['debra']

def compare(label, ibr_val, debra_val, unit='', lower_better=True, fmt=','):
    """Print comparison row with winner indicator."""
    if isinstance(ibr_val, float):
        ibr_str = f"{ibr_val:.2f}"
        debra_str = f"{debra_val:.2f}"
    else:
        ibr_str = f"{ibr_val:{fmt}}"
        debra_str = f"{debra_val:{fmt}}"
    
    if lower_better:
        ibr_wins = ibr_val < debra_val
    else:
        ibr_wins = ibr_val > debra_val
    
    ibr_mark = '✓' if ibr_wins else ' '
    debra_mark = ' ' if ibr_wins else '✓'
    
    print(f"{label:<30} {ibr_str:>15}{unit} {ibr_mark}  {debra_str:>15}{unit} {debra_mark}")

print("=" * 80)
print("IBR vs DEBRA+ COMPARISON")
print("=" * 80)
print(f"{'Metric':<30} {'IBR':>16}    {'DEBRA+':>16}")
print("-" * 80)

print("\n### Throughput ###")
compare("Operations/sec", ibr.ops_per_sec, debra.ops_per_sec, "", lower_better=False)
compare("Total operations", ibr.operations, debra.operations, "", lower_better=False)

print("\n### Memory ###")
compare("Peak allocated", ibr.peak_allocated_bytes // 1024, debra.peak_allocated_bytes // 1024, " KB")
compare("Peak pending nodes", ibr.peak_pending_count, debra.peak_pending_count, "")
compare("Peak pending bytes", ibr.peak_pending_bytes, debra.peak_pending_bytes, "")
compare("Bound utilization", ibr.bound_utilization * 100, debra.bound_utilization * 100, "%")

print("\n### Reclamation ###")
compare("Total retired", ibr.total_retired, debra.total_retired, "", lower_better=False)
compare("Reclaim latency P50", ibr.reclaim_latency_p50_ns / 1000, debra.reclaim_latency_p50_ns / 1000, " µs")
compare("Reclaim latency P99", ibr.reclaim_latency_p99_ns / 1000, debra.reclaim_latency_p99_ns / 1000, " µs")

print("\n### Epochs ###")
compare("Epoch advances", ibr.epoch_advances, debra.epoch_advances, "", lower_better=False)
compare("Max epoch lag", ibr.safe_epoch_lag_max, debra.safe_epoch_lag_max, " epochs")

print("\n### Stall Handling ###")
compare("Stall count", ibr.stall_count, debra.stall_count, "")
compare("Stall time", ibr.stall_time_ns / 1e6, debra.stall_time_ns / 1e6, " ms")
print(f"{'Neutralizations':<30} {'N/A':>16}    {debra.neutralization_count:>16,}")
print(f"{'Operation restarts':<30} {'N/A':>16}    {debra.restart_count:>16,}")

## 6. Memory Bound Analysis

In [ ]:
print("### Memory Bound Analysis ###\n")

T = TEST_CONFIG['num_threads']
R = 64

ibr_theoretical = 3 * T * T * R
debra_theoretical = 3 * T * R

print(f"Configuration: T={T} threads, R={R} retire threshold\n")

print(f"IBR (O(T²R)):")
print(f"  Theoretical max: {ibr_theoretical:,} nodes")
print(f"  Actual peak: {ibr.peak_pending_count:,} nodes")
print(f"  Utilization: {ibr.peak_pending_count / ibr_theoretical:.1%}")
print(f"  Headroom: {ibr_theoretical - ibr.peak_pending_count:,} nodes")

print(f"\nDEBRA+ (O(TR)):")
print(f"  Theoretical max: {debra_theoretical:,} nodes")
print(f"  Actual peak: {debra.peak_pending_count:,} nodes")
print(f"  Utilization: {debra.peak_pending_count / debra_theoretical:.1%}")
print(f"  Headroom: {debra_theoretical - debra.peak_pending_count:,} nodes")

print(f"\nComparison:")
memory_savings = (ibr.peak_pending_bytes - debra.peak_pending_bytes) / ibr.peak_pending_bytes * 100
print(f"  DEBRA+ uses {memory_savings:.0f}% less pending memory")
print(f"  DEBRA+ theoretical bound is {T}x tighter")

if ibr.peak_pending_count > debra_theoretical:
    print(f"\n⚠️ IBR exceeded DEBRA+ theoretical bound!")
    print(f"   IBR would need DEBRA+ to guarantee memory safety here.")

## 7. Stall Handling Effectiveness

In [ ]:
print("### Stall Handling Comparison ###\n")

print("IBR Approach: Cooperative")
print(f"  Stalled threads block epoch advancement")
print(f"  Stall events: {ibr.stall_count}")
print(f"  Total blocked time: {ibr.stall_time_ns / 1e6:.0f} ms")
print(f"  Max epoch lag: {ibr.safe_epoch_lag_max} epochs")

print(f"\nDEBRA+ Approach: Neutralization")
print(f"  Stalled threads are neutralized via signals")
print(f"  Neutralizations sent: {debra.neutralization_count}")
print(f"  Operation restarts: {debra.restart_count}")
print(f"  Stall events (post-neutralization): {debra.stall_count}")
print(f"  Max epoch lag: {debra.safe_epoch_lag_max} epochs")

# Effectiveness metrics
if ibr.stall_count > 0:
    neutralization_effectiveness = (ibr.stall_count - debra.stall_count) / ibr.stall_count * 100
    lag_reduction = (ibr.safe_epoch_lag_max - debra.safe_epoch_lag_max) / ibr.safe_epoch_lag_max * 100
    
    print(f"\nDEBRA+ Effectiveness:")
    print(f"  Stall reduction: {neutralization_effectiveness:.0f}%")
    print(f"  Epoch lag reduction: {lag_reduction:.0f}%")
    
    restart_overhead = debra.restart_count / debra.operations * 100
    print(f"  Restart overhead: {restart_overhead:.3f}% of operations")

## 8. Visualizations

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    PLOTTING_AVAILABLE = True
except ImportError:
    PLOTTING_AVAILABLE = False
    print("matplotlib not available")

In [ ]:
if PLOTTING_AVAILABLE:
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    
    # 1. Memory usage comparison
    ax1 = axes[0, 0]
    schemes = ['IBR', 'DEBRA+']
    pending_counts = [ibr.peak_pending_count, debra.peak_pending_count]
    theoretical = [ibr_theoretical, debra_theoretical]
    x = np.arange(2)
    width = 0.35
    ax1.bar(x - width/2, pending_counts, width, label='Actual Peak', color='steelblue')
    ax1.bar(x + width/2, theoretical, width, label='Theoretical Max', color='lightcoral', alpha=0.7)
    ax1.set_xticks(x)
    ax1.set_xticklabels(schemes)
    ax1.set_ylabel('Pending Nodes')
    ax1.set_title('Memory Bound Comparison')
    ax1.legend()
    ax1.set_yscale('log')
    
    # 2. Throughput comparison
    ax2 = axes[0, 1]
    throughputs = [ibr.ops_per_sec, debra.ops_per_sec]
    colors = ['coral', 'mediumseagreen']
    ax2.bar(schemes, throughputs, color=colors)
    ax2.set_ylabel('Operations/sec')
    ax2.set_title('Throughput Comparison')
    for i, v in enumerate(throughputs):
        ax2.text(i, v + 1000, f'{v:,.0f}', ha='center')
    
    # 3. Latency comparison
    ax3 = axes[0, 2]
    latencies_p50 = [ibr.reclaim_latency_p50_ns / 1000, debra.reclaim_latency_p50_ns / 1000]
    latencies_p99 = [ibr.reclaim_latency_p99_ns / 1000, debra.reclaim_latency_p99_ns / 1000]
    x = np.arange(2)
    ax3.bar(x - width/2, latencies_p50, width, label='P50', color='lightgreen')
    ax3.bar(x + width/2, latencies_p99, width, label='P99', color='salmon')
    ax3.set_xticks(x)
    ax3.set_xticklabels(schemes)
    ax3.set_ylabel('Latency (µs)')
    ax3.set_title('Reclamation Latency')
    ax3.legend()
    
    # 4. Epoch lag comparison
    ax4 = axes[1, 0]
    epoch_lags = [ibr.safe_epoch_lag_max, debra.safe_epoch_lag_max]
    colors = ['indianred' if lag > 10 else 'gold' if lag > 5 else 'mediumseagreen' for lag in epoch_lags]
    ax4.bar(schemes, epoch_lags, color=colors)
    ax4.set_ylabel('Max Epoch Lag')
    ax4.set_title('Epoch Lag (lower is better)')
    ax4.axhline(y=10, color='red', linestyle='--', alpha=0.5, label='Warning threshold')
    
    # 5. Stall handling
    ax5 = axes[1, 1]
    stall_data = {
        'IBR': [ibr.stall_count, 0],
        'DEBRA+': [debra.stall_count, debra.neutralization_count],
    }
    x = np.arange(2)
    ax5.bar(x - width/2, [ibr.stall_count, debra.stall_count], width, 
            label='Unhandled Stalls', color='indianred')
    ax5.bar(x + width/2, [0, debra.neutralization_count], width,
            label='Neutralizations', color='steelblue')
    ax5.set_xticks(x)
    ax5.set_xticklabels(schemes)
    ax5.set_ylabel('Count')
    ax5.set_title('Stall Handling')
    ax5.legend()
    
    # 6. Summary radar chart
    ax6 = axes[1, 2]
    categories = ['Throughput', 'Memory\nEfficiency', 'Latency', 'Stall\nHandling', 'Simplicity']
    
    # Normalize scores (0-10)
    ibr_scores = [8, 5, 6, 3, 10]
    debra_scores = [7.5, 9, 8, 9, 6]
    
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    ibr_scores_plot = ibr_scores + [ibr_scores[0]]
    debra_scores_plot = debra_scores + [debra_scores[0]]
    angles += angles[:1]
    
    ax6 = fig.add_subplot(2, 3, 6, projection='polar')
    ax6.plot(angles, ibr_scores_plot, 'o-', linewidth=2, label='IBR', color='coral')
    ax6.fill(angles, ibr_scores_plot, alpha=0.25, color='coral')
    ax6.plot(angles, debra_scores_plot, 'o-', linewidth=2, label='DEBRA+', color='steelblue')
    ax6.fill(angles, debra_scores_plot, alpha=0.25, color='steelblue')
    ax6.set_xticks(angles[:-1])
    ax6.set_xticklabels(categories)
    ax6.set_title('Overall Comparison')
    ax6.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    plt.tight_layout()
    plt.savefig('smr_comparison_charts.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Charts saved to smr_comparison_charts.png")

## 9. Decision Guide

In [ ]:
print("### When to Use Each Scheme ###\n")

print("Choose IBR when:")
print("  ✓ Running on Windows (DEBRA+ falls back to IBR anyway)")
print("  ✓ Workload has minimal blocking operations")
print("  ✓ Thread count is low (< 8)")
print("  ✓ Memory bound is not a concern")
print("  ✓ Simplicity is preferred over optimal memory usage")
print("  ✓ No tolerance for operation restarts")

print("\nChoose DEBRA+ when:")
print("  ✓ Threads may stall (I/O, page faults, GC)")
print("  ✓ Strict memory bounds required")
print("  ✓ High thread count (8+)")
print("  ✓ Running on POSIX (Linux, macOS)")
print("  ✓ Occasional operation restarts are acceptable")
print("  ✓ Predictable memory usage is important")

print("\n### Your Workload Analysis ###")

stall_prone = ibr.stall_count > 5
memory_pressure_high = ibr.bound_utilization > 0.5
high_thread_count = T >= 8

recommendation_score = sum([
    stall_prone * 3,
    memory_pressure_high * 2,
    high_thread_count * 1,
])

print(f"  Stall-prone workload: {'Yes' if stall_prone else 'No'}")
print(f"  High memory pressure: {'Yes' if memory_pressure_high else 'No'}")
print(f"  High thread count: {'Yes' if high_thread_count else 'No'}")

if recommendation_score >= 4:
    print(f"\n🔵 RECOMMENDATION: Use DEBRA+")
    print(f"   Your workload will benefit from neutralization and tighter memory bounds.")
elif recommendation_score >= 2:
    print(f"\n🟡 RECOMMENDATION: Consider DEBRA+")
    print(f"   DEBRA+ offers modest improvements; test both in production.")
else:
    print(f"\n🟢 RECOMMENDATION: Use IBR")
    print(f"   Your workload doesn't require DEBRA+'s extra complexity.")

## 10. Export Results

In [ ]:
import json
from dataclasses import asdict

comparison_report = {
    'config': TEST_CONFIG,
    'ibr': asdict(ibr) if hasattr(ibr, '__dataclass_fields__') else vars(ibr),
    'debra': asdict(debra) if hasattr(debra, '__dataclass_fields__') else vars(debra),
    'analysis': {
        'memory_savings_pct': memory_savings if 'memory_savings' in dir() else 0,
        'stall_prone': stall_prone,
        'memory_pressure_high': memory_pressure_high,
        'recommendation': 'debra' if recommendation_score >= 4 else 'ibr' if recommendation_score < 2 else 'either',
    }
}

with open('smr_comparison_report.json', 'w') as f:
    json.dump(comparison_report, f, indent=2, default=str)

print("Report exported to smr_comparison_report.json")

## Summary

This notebook compared IBR and DEBRA+ across:

| Aspect | IBR | DEBRA+ |
|--------|-----|--------|
| Memory Bound | O(T²R) | O(TR) |
| Stall Handling | Blocks | Neutralizes |
| Complexity | Simple | Moderate |
| Platforms | All | POSIX only* |
| Restarts | Never | Sometimes |

*DEBRA+ falls back to IBR on Windows

For module-specific analysis, see:
- `memory_performance_analysis.ipynb` - Allocation profiling
- `smr_performance_analysis.ipynb` - SMR-specific metrics